In [55]:
import datetime
import os
import json
from datetime import timezone
from tzlocal import get_localzone
from typing import TypedDict, Annotated, List, Optional
import operator

from dotenv import load_dotenv;
from pydantic import  BaseModel, Field

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError



from langchain_core.tools import tool
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.sqlite import SqliteSaver


from IPython.display import Image, display
from dateutil.parser import parse, ParserError

In [56]:
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY") 

SCOPES = ["https://www.googleapis.com/auth/calendar"]

if not all([groq_api_key, tavily_api_key]):
    raise ValueError("One or more required API keys (GROQ, TAVILY) are missing from the .env file!")


llm = ChatGroq(model="llama3-70b-8192", api_key=groq_api_key, temperature=0)
print("Groq LLM (Llama3-70b) configured and ready.")

Groq LLM (Llama3-70b) configured and ready.


In [57]:
class CalendarEvent(BaseModel):
    """Schema for a single, richly detailed calendar event."""
    id: str = Field(description="The unique ID of the calendar event.")
    summary: str = Field(description="The title or summary of the calendar event.")
    start_time: str = Field(description="The start time of the event in ISO format.")
    source: str = Field(description="The source of the calendar, either 'Google' or 'Apple'.")
    description: Optional[str] = Field(default=None, description="The detailed description or notes for the event.")
    attendees: Optional[List[str]] = Field(default=None, description="A list of email addresses of the event attendees.")
    location: Optional[str] = Field(default=None, description="The physical location of the event (e.g., an address or room name).")
    video_conference_link: Optional[str] = Field(default=None, description="The link for the Google Meet video conference, if available.")
    attachments: Optional[List[str]] = Field(default=None, description="A list of Google Drive attachments, formatted as 'title (link)'.")



def _get_google_credentials():
    """Gets valid Google API credentials."""
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    return creds



def _fetch_google_events(start_date: datetime.datetime, end_date: datetime.datetime) -> List[dict]:
    """
    Internal function to fetch events from Google Calendar, now including ALL rich details:
    attendees, description, location, conference links, and attachments.
    """
    try:
        creds = _get_google_credentials()
        service = build("calendar", "v3", credentials=creds)
        
        events_result = service.events().list(
            calendarId="primary",
            timeMin=start_date.isoformat(),
            timeMax=end_date.isoformat(),
            maxResults=250,
            singleEvents=True,
            orderBy="startTime"
        ).execute()
        
        events = events_result.get("items", [])
        if not events: return []

        output_events = []
        for event in events:
            start = event["start"].get("dateTime", event["start"].get("date"))
            
            description = event.get("description")
            
            attendee_emails = [att.get("email") for att in event.get("attendees", []) if att.get("email")]

            location = event.get("location")
            
            video_conference_link = None
            if "conferenceData" in event and "entryPoints" in event["conferenceData"]:
                for entry_point in event["conferenceData"]["entryPoints"]:
                    if entry_point.get("entryPointType") == "video":
                        video_conference_link = entry_point.get("uri")
                        break 
            
            attachment_info = []
            if "attachments" in event:
                for attachment in event["attachments"]:
                    title = attachment.get("title", "Untitled Attachment")
                    link = attachment.get("fileUrl", "#")
                    attachment_info.append(f"{title} ({link})")

            event_model = CalendarEvent(
                id=event.get("id"),
                summary=event.get("summary", "No Title"), 
                start_time=start, 
                source="Google",
                description=description,
                attendees=attendee_emails if attendee_emails else None,
                location=location,
                video_conference_link=video_conference_link,
                attachments=attachment_info if attachment_info else None 
            )
            output_events.append(event_model.model_dump())
        return output_events
    except Exception as e:
        print(f"!!! Google Calendar Read Error: {e}")
        return [{"error": f"An error occurred while reading Google Calendar: {e}"}]


In [58]:
@tool
def list_upcoming_events(limit: Optional[int] = 15) -> str:
    """
    Lists the user's next upcoming events from today onwards from Google Calendar.
    Use this tool for general queries like 'list my events' or 'what's next?'.
    This tool does NOT take a specific date parameter.
    """
    print(f"--- Tool: list_upcoming_events called with limit={limit} ---")
    
    now = datetime.datetime.now(timezone.utc)
    search_end = now + datetime.timedelta(days=90) 

    google_events = _fetch_google_events(start_date=now, end_date=search_end)
    
    if not google_events or all('error' in e for e in google_events):
        return json.dumps([{"message": "No upcoming events found."}])

    valid_events = [e for e in google_events if 'error' not in e]
    
    if not limit: limit = 15
    events_to_return = valid_events[:limit]
    
    return json.dumps(events_to_return, indent=2)


@tool
def get_events_for_date(target_date: str) -> str:
    """
    Gets all Google Calendar events for a specific given date, including rich
    details like the description, attendees, location, and conference links.
    Use this tool when the user asks about a particular date (e.g., 'tomorrow', 'August 22, 2025').
    This tool REQUIRES the 'target_date' parameter in 'YYYY-MM-DD' format.
    """
    print(f"--- Tool: get_events_for_date called with date='{target_date}' ---")
    
    try:
        parsed_target_date = parse(target_date).date()
    except ParserError:
        return json.dumps([{"error": f"Invalid date format provided: {target_date}"}])


    start_of_day = datetime.datetime.combine(parsed_target_date, datetime.time.min).replace(tzinfo=timezone.utc)
    end_of_day = datetime.datetime.combine(parsed_target_date, datetime.time.max).replace(tzinfo=timezone.utc)
    
    google_events = _fetch_google_events(start_date=start_of_day, end_date=end_of_day)
    valid_events = [e for e in google_events if 'error' not in e]
    
    if not valid_events:
        return json.dumps([])

    return json.dumps(valid_events, indent=2)


@tool
def tavily_search(query: str) -> str:
    """
    A search engine tool to find real-time information online.
    Use this for any questions about weather, news, current events, facts,
    or general knowledge that is not related to the user's personal calendar.
    Do NOT claim you cannot answer these types of questions.
    """
    print(f"--- Tool: tavily_search called with query: '{query}' ---")
    try:
        search = TavilySearch(max_results=3, api_key=tavily_api_key)
        results = search.invoke(query)
        
        if not results:
            return "No results found from web search."

        
        return results

        
        
    except Exception as e:
        print(f"!!! TAVILY SEARCH FAILED: {e}")
        return f"Error occurred during web search: {e}"



@tool
def create_google_event(
    summary: str, 
    start_time_iso: str, 
    end_time_iso: str, 
    description: Optional[str] = None, 
    location: Optional[str] = None, 
    attendees: Optional[List[str]] = None
) -> str:
    """
    Creates a new event on the user's primary Google Calendar after checking for conflicts.

    Args:
        summary (str): The title or summary of the event.
        start_time_iso (str): The start time in ISO 8601 format (e.g., '2025-08-18T15:00:00').
        end_time_iso (str): The end time in ISO 8601 format (e.g., '2025-08-18T16:00:00').
        description (Optional[str]): A detailed description or notes for the event.
        location (Optional[str]): The physical location or address of the event.
        attendees (Optional[List[str]]): A list of attendee email addresses to invite.
    """
    print(f"--- Tool: create_google_event called with: summary='{summary}', start='{start_time_iso}', end='{end_time_iso}', description='{description}', location='{location}', attendees='{attendees}' ---")
    
    try:

        start_time_obj = parse(start_time_iso)
        end_time_obj = parse(end_time_iso)
        
        local_tz = get_localzone()

        if start_time_obj.tzinfo is None:
            start_time_obj = start_time_obj.replace(tzinfo=local_tz)
        if end_time_obj.tzinfo is None:
            end_time_obj = end_time_obj.replace(tzinfo=local_tz)

        conflicting_events = _fetch_google_events(start_date=start_time_obj, end_date=end_time_obj)
        valid_conflicts = [e for e in conflicting_events if 'error' not in e]

        if valid_conflicts:
            conflict_summary = valid_conflicts[0].get('summary', 'an existing event')
            print(f"--- Failure: Event conflict found with '{conflict_summary}' ---")
            return f"Error: Cannot create event. There is a conflicting event at that time: '{conflict_summary}'."

        creds = _get_google_credentials()
        service = build("calendar", "v3", credentials=creds)
        
        local_timezone = str(get_localzone())
        
        event_body = {
            'summary': summary,
            'start': {'dateTime': start_time_iso, 'timeZone': local_timezone},
            'end': {'dateTime': end_time_iso, 'timeZone': local_timezone},
        }

        if description:
            event_body['description'] = description
        
        if location:
            event_body['location'] = location
        
        if attendees:
            event_body['attendees'] = [{'email': email} for email in attendees]

        created_event = service.events().insert(calendarId='primary', body=event_body).execute()
        
        print(f"--- Success: Event created. ID: {created_event.get('id')} ---")
        return f"Event '{summary}' was created successfully with all provided details."

    except Exception as e:
        print(f"!!! Google Calendar Write Error: {e}")
        return f"An error occurred while creating the event in Google Calendar: {e}"



@tool
def delete_google_event(event_id: str, summary: str) -> str:
    """
    Deletes an event from the user's primary Google Calendar using its unique ID.

    To use this tool, you must first find the event and its 'id' by using
    `get_events_for_date` or `list_upcoming_events`.

    Args:
        event_id (str): The unique ID of the event to be deleted.
        summary (str): The summary/title of the event, used for the confirmation message.
    """
    print(f"--- Tool: delete_google_event called for event ID: {event_id} ---")
    try:
        creds = _get_google_credentials()
        service = build("calendar", "v3", credentials=creds)

        service.events().delete(
            calendarId='primary',
            eventId=event_id
        ).execute()
        
        print(f"--- Success: Event '{summary}' deleted. ---")
        return f"The event '{summary}' was successfully deleted from the calendar."

    except Exception as e:
        print(f"!!! Google Calendar Delete Error: {e}")
        return f"An error occurred while trying to delete the event: {e}"


@tool
def get_calendar_summary(time_range: str) -> str:
    """
    Provides a summary of Google Calendar events for a given time range.
    Use this for questions like "what's my schedule this week?" or "give me my weekly report".

    Args:
        time_range (str): The time range to summarize. Supported values: "today", "tomorrow", "this week", "next week".
    """
    print(f"--- Tool: get_calendar_summary called for range: {time_range} ---")
    today = datetime.date.today()
    
    if time_range == "today":
        start_date = today
        end_date = today
    elif time_range == "tomorrow":
        start_date = today + datetime.timedelta(days=1)
        end_date = start_date
    elif time_range == "this week":
        start_date = today - datetime.timedelta(days=today.weekday())
        end_date = start_date + datetime.timedelta(days=6)
    elif time_range == "next week":
        start_date = today - datetime.timedelta(days=today.weekday()) + datetime.timedelta(days=7)
        end_date = start_date + datetime.timedelta(days=6)
    else:
        return json.dumps([{"error": "Unsupported time range. Please use 'today', 'tomorrow', 'this week', or 'next week'."}])

    start_datetime = datetime.datetime.combine(start_date, datetime.time.min).replace(tzinfo=timezone.utc)
    end_datetime = datetime.datetime.combine(end_date, datetime.time.max).replace(tzinfo=timezone.utc)
    
    events = _fetch_google_events(start_date=start_datetime, end_date=end_datetime)
    
    return json.dumps(events, indent=2)



@tool
def update_google_event(
    event_id: str,
    summary: Optional[str] = None,
    start_time_iso: Optional[str] = None,
    end_time_iso: Optional[str] = None,
    description: Optional[str] = None,
    location: Optional[str] = None,
    attendees: Optional[List[str]] = None
) -> str:
    """
    Updates an existing event on the user's primary Google Calendar using its unique ID.
    To use this tool, you must first find the event and its 'id'.
    Only provide the fields that are being changed.
    If only start_time_iso is provided, the original duration will be maintained.
    """
    print(f"--- Tool: update_google_event called for event ID: {event_id} ---")
    try:
        creds = _get_google_credentials()
        service = build("calendar", "v3", credentials=creds)

        event = service.events().get(calendarId='primary', eventId=event_id).execute()

        if start_time_iso and not end_time_iso:
            start_dt_obj = parse(current_event["start"].get("dateTime"))
            end_dt_obj = parse(current_event["end"].get("dateTime"))
            original_duration = end_dt_obj - start_dt_obj
            
            new_start_dt_obj = parse(start_time_iso)
            new_end_dt_obj = new_start_dt_obj + original_duration
            end_time_iso = new_end_dt_obj.isoformat()

        update_body = {}
        local_timezone = str(get_localzone())

        if summary:
            update_body['summary'] = summary
        if description:
            update_body['description'] = description
        if location:
            update_body['location'] = location
        if attendees is not None: 
            update_body['attendees'] = [{'email': email} for email in attendees]
        if start_time_iso:
            update_body['start'] = {'dateTime': start_time_iso, 'timeZone': local_timezone}
        if end_time_iso:
            update_body['end'] = {'dateTime': end_time_iso, 'timeZone': local_timezone}

        if not update_body:
            return "Error: No update information was provided."

        updated_event = service.events().patch(
            calendarId='primary',
            eventId=event_id,
            body=update_body
        ).execute()
        
        final_summary = updated_event.get('summary')
        start = parse(updated_event["start"].get("dateTime")).strftime("%Y-%m-%d %H:%M")
        end = parse(updated_event["end"].get("dateTime")).strftime("%Y-%m-%d %H:%M")

        print(f"--- Success: Event '{final_summary}' updated. ---")
        return f"Success! Event '{final_summary}' was updated to take place from {start} to {end}."

    except HttpError as err:
        error_reason = err.reason if hasattr(err, 'reason') else 'Unknown API Error'
        print(f"!!! Google API HTTP Error: {err.resp.status} - {error_reason} !!!")
        if err.resp.status == 404:
            return f"Error: The event with ID '{event_id}' was not found. It may have been deleted."
        return f"An error occurred with the Google Calendar API: {error_reason}"
    except Exception as e:
        print(f"!!! Google Calendar Generic Update Error: {e}")
        return f"An unexpected error occurred while trying to update the event: {e}"



tools = [list_upcoming_events, get_events_for_date, create_google_event, delete_google_event, get_calendar_summary, update_google_event, tavily_search]
print(f"Defined {len(tools)} tools for the agent: {[t.name for t in tools]}")

Defined 7 tools for the agent: ['list_upcoming_events', 'get_events_for_date', 'create_google_event', 'delete_google_event', 'get_calendar_summary', 'update_google_event', 'tavily_search']


In [59]:
class AgentState(TypedDict):
    messages: Annotated[list, operator.add]


llm_with_tools = llm.bind_tools(tools)
tool_node = ToolNode(tools)


system_prompt_template = """
You are a multi-talented personal assistant. Today's date is {current_date}.

**CRITICAL BEHAVIORAL RULES:**
1.  **!!! NON-NEGOTIABLE RULE: TOOL USE IS MANDATORY !!!** For ANY factual question, especially about the user's calendar (schedule, event, free, busy, a date, a time) or the real world (weather, news), you are FORBIDDEN from answering from memory. Your ONLY valid first step is to call a tool (get_events_for_date, tavily_search, etc.). Do NOT say "Let me check" and then fail to use a tool. Immediately call the tool.
2.  **CONVERSATION-FIRST FOR CHAT:** For simple greetings, social chat, or opinions, you MUST respond conversationally without using any tools.

Your capabilities are:
1.  **General Conversation:** You can chat about any topic.
2.  **Calendar Management:** You can read, write and delete to the user's Google Calendar.
3.  **Web Search:** For questions about real-time information (like weather, news), you MUST use the `tavily_search` tool. Do not claim you cannot answer.

**Core Instructions & Tool Rules:**

1.  **The Golden Rule of Dates:** - You MUST ALWAYS use today's date ({current_date}) as the absolute reference point for ANY calculation involving relative dates.
    - NEVER use a date from a previous turn in the conversation as your reference.

2.  **Resolving Dates and Ranges:** - You must resolve all relative date expressions into an absolute 'YYYY-MM-DD' format before calling a tool.
    - 'Next [day of week]' refers to the nearest upcoming day of that week.
    - Phrases like "in X days" or "next X days" ALWAYS define a date range starting from today.

3.  **Choosing and Using Tools Correctly:**
    - **For GENERAL requests** (e.g., "what are my next events?"), you MUST use the `list_upcoming_events` tool.
    - **For SPECIFIC day requests** (e.g., "what's on for tomorrow?"), you MUST call the `get_events_for_date` tool. **After this tool returns a result, you must immediately follow the procedure in Rule 6.**
    - **For DATE RANGE requests** (e.g., "what's happening in the next 3 days?"), you MUST call `get_events_for_date` multiple times. **After synthesizing the results, you must immediately follow the procedure in Rule 6.**
    - **For summary requests over a period of time** ("this week", "next week"), use the `get_calendar_summary` tool. **After this tool returns a result, you must immediately follow the procedure in Rule 6.**
    - **SINGLE-ACTION PRINCIPLE:** After a calendar search tool (`get_events_for_date` or `list_upcoming_events`) successfully returns information, your IMMEDIATE next action is to present that information to the user. DO NOT call another search tool unless the user asks a new, different question.


4.  **Writing to the Calendar (`create_google_event`):**
    - Before creating an event, you MUST have three pieces of information: a summary (title), a start time, and a duration/end time. If the user does not provide a duration (e.g., "for 1 hour"), you MUST ask a clarifying question like "And for how long should I schedule that?". Do not assume a default duration
    - If the tool returns a conflict error, you MUST inform the user and ask for a different time.

5.  **Deleting from the Calendar (`delete_google_event` - CRITICAL 3-STEP PROCEDURE):**
    You MUST follow this three-step "Search, Confirm, Execute" procedure to delete an event.
    - **Step 1: SEARCH.** When the user asks to delete an event (e.g., "cancel my meeting"), your FIRST and ONLY action is to use `get_events_for_date` or `list_upcoming_events` to find the exact event. You must obtain its `id`, `summary`, and `start_time`.
    - **Step 2: CONFIRM.** After the search tool returns the event details, you MUST ask the user for confirmation. Your response MUST be in a format like: "I found '[Summary]' on [Date] at [Time]. Are you sure you want to delete it?". **DO NOT call `delete_google_event` in this step.**
    - **Step 3: EXECUTE.** ONLY AFTER the user explicitly confirms (e.g., "Yes", "Correct", "Please do"), you will then make a NEW call to the `delete_google_event` tool, using the `id` you found in Step 1.

6.  **Update Event (`update_google_event` - 2-Step Process):**
    - **Step 1:  Search:** When a user wants to change an event (e.g., "move my meeting to 3 PM"), your FIRST action is to use a read tool (`get_events_for_date`) to find the event and get its `id`.
    - **Step 2:  Execute:** Once you have the `id` and know exactly what needs to be changed, call `update_google_event` with the `event_id` and the new information.
"""

def agent_node(state):
    today_str = datetime.date.today().isoformat()
    formatted_prompt = system_prompt_template.format(current_date=today_str)
    
    current_messages = state['messages']
    if not current_messages or not isinstance(current_messages[0], SystemMessage):
        messages_with_prompt = [SystemMessage(content=formatted_prompt)] + current_messages
    else:
        current_messages[0] = SystemMessage(content=formatted_prompt)
        messages_with_prompt = current_messages

    response = llm_with_tools.invoke(messages_with_prompt)
    return {"messages": [response]}


def should_continue(state):
    if state["messages"][-1].tool_calls:
        return "tools"
    return END

In [60]:
workflow = StateGraph(AgentState)

workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)

workflow.set_entry_point("agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")


In [61]:
db_path = os.path.abspath("conversations.sqlite")

print(f"Using database at: {db_path}")

with SqliteSaver.from_conn_string(db_path) as memory:
    
    app = workflow.compile(checkpointer=memory)

    print("\nGraph is compiled with persistent memory. Ready for interaction.")

    config = {"configurable": {"thread_id": "berky_alkn13"}}

    print("\nYour persistent assistant is ready. You can now start chatting.")
    print("   Type 'quit' or 'exit' to end the conversation.")
    print("   To start a new conversation, restart the script and change the 'thread_id'.")
    print("-" * 50)

    while True:
        try:
            user_input = input("You: ")
            if user_input.lower() in ["quit", "exit"]:
                print("Assistant: Goodbye!")
                break

            print(f"You: {user_input}")

            response = app.invoke(
                {"messages": [HumanMessage(content=user_input)]},config
            )
            
            final_response = response["messages"][-1]
            print(f"Assistant: {final_response.content}")
        
        except Exception as e:
            print(f"\n--- An Error Occurred ---")
            print(f"Error: {e}")
            print("Please try again.")


Using database at: /Users/berkayalkan/Lecture/CMPE411/gitHub/ai-personal-assistant/conversations.sqlite

Graph is compiled with persistent memory. Ready for interaction.

Your persistent assistant is ready. You can now start chatting.
   Type 'quit' or 'exit' to end the conversation.
   To start a new conversation, restart the script and change the 'thread_id'.
--------------------------------------------------
You: Hi
Assistant: Hi! How can I help you today?
You: Can you check do I have any event on September 1 ?
--- Tool: get_events_for_date called with date='2025-09-01' ---
Assistant: You have an event on September 1, 2025: "Meeting with Friends" at 20:00 at Raw Coffee Konyaalti.
You: Perfect, how long it will take ?
Assistant: The "Meeting with Friends" event is scheduled to last for 2 hours and 40 minutes, from 20:00 to 22:40.
You: That is problem becasue I sleep at 10 PM everyday. Can you move starting time to 6 PM please ?
--- Tool: update_google_event called for event ID: gslbr